In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
import warnings
warnings.filterwarnings('ignore')

In [2]:
import logging

class GERdailyZuwachs(scrapy.Spider):
    name = 'corona_gerdaily'
    start_urls = ['https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_Deutschland/Statistik']

    def parse(self, response):
        for bundesland in response.css('table.wikitable > tbody > tr'):    #.zebra.toptextcells.mw-collapsible.mw-made-collapsible

                yield{
                    
                  'BW' : bundesland.xpath('td[2]//text()').extract_first(),
                  'BY' : bundesland.xpath('td[3]//text()').extract_first(),
                  'BE' : bundesland.xpath('td[4]//text()').extract_first(),
                  'BB' : bundesland.xpath('td[5]//text()').extract_first(),
                  'HB' : bundesland.xpath('td[6]//text()').extract_first(),
                  'HH' : bundesland.xpath('td[7]//text()').extract_first(),
                  'HE' : bundesland.xpath('td[8]//text()').extract_first(),
                  'MV' : bundesland.xpath('td[9]//text()').extract_first(),
                  'NI' : bundesland.xpath('td[10]//text()').extract_first(),
                  'NW' : bundesland.xpath('td[11]//text()').extract_first(),
                  'RP' : bundesland.xpath('td[12]//text()').extract_first(),
                  'SL' : bundesland.xpath('td[13]//text()').extract_first(),
                  'SN' : bundesland.xpath('td[14]//text()').extract_first(),
                  'ST' : bundesland.xpath('td[15]//text()').extract_first(),
                  'SH' : bundesland.xpath('td[16]//text()').extract_first(),
                  'TH' : bundesland.xpath('td[17]//text()').extract_first(),
                  'Gesamt' : bundesland.xpath('td[18]//text()').extract_first(),
               }

process = CrawlerProcess({
'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
'FEED_FORMAT': 'csv',
'FEED_URI': 'de_zuwachsneu.csv'
})

#process.crawl(GERdailyZuwachs)
#process.start()

2020-10-30 12:46:22 [scrapy.utils.log] INFO: Scrapy 2.3.0 started (bot: scrapybot)
2020-10-30 12:46:22 [scrapy.utils.log] INFO: Versions: lxml 4.5.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 20.3.0, Python 3.7.7 (default, May  6 2020, 11:45:54) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1h  22 Sep 2020), cryptography 3.1.1, Platform Windows-10-10.0.19041-SP0
2020-10-30 12:46:22 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor


In [3]:
import pandas as pd
import numpy as np
de_zuwachs = pd.read_csv('de_zuwachs4.csv', skiprows = range(1,547), nrows = 105 , thousands = '.' ) #, dtype={'BW': int } )
de_zuwachs['Datum'] = pd.date_range(start='24/2/2020', periods=len(de_zuwachs), freq='D')

#löst das Problem mit dem falschen Datum
first_col = de_zuwachs.pop('Datum')
de_zuwachs.insert(0, 'Datum', first_col)

#Ändert die NAN und - zu 0
de_zuwachs = de_zuwachs.fillna(0)
de_zuwachs = de_zuwachs.replace('—',0)

In [4]:
de_tode = pd.read_csv('de_zuwachs.csv', skiprows = range(1,735), nrows = 93, thousands = '.') 

de_tode['Datum'] = pd.date_range(start='3/9/2020', periods=len(de_tode), freq='D')
first_col = de_tode.pop('Datum')
de_tode.insert(0, 'Datum', first_col)

#Ändert die NAN und - zu 0
de_tode = de_tode.fillna(0)
de_tode = de_tode.replace('–',0)

In [5]:
welt_zuwachs = pd.read_csv('WHO-COVID-19-global-data.csv')
welt_zuwachs.drop(['Country_code','WHO_region'], axis=1, inplace=True)
welt_zuwachs['Date_reported'] = welt_zuwachs['Date_reported'].replace('T00:00:00Z'," ",regex = True)
#welt_zuwachs.head()
arraywelt = ["Germany"]
de_zuwachs = welt_zuwachs.loc[welt_zuwachs['Country'].isin(arraywelt)]

In [39]:
kreise_zuwachs = pd.read_csv('COVID-19 Cases Germany_1591375436907.csv')
kreise_zuwachs.drop(['relative_case_changes','fb_id','cases_per_population','relative_death_changes','bundesland_ags','kreis_ags','kreis_nuts','kreis_name','deaths_per_population','fb_datetime'], axis=1, inplace=True)
kreise_zuwachs = kreise_zuwachs.fillna(0)
kreise_zuwachs['deaths'] = kreise_zuwachs['deaths'].astype(int)
kreise_zuwachs['publication_datetime'] = kreise_zuwachs['publication_datetime'].str[:11]

In [7]:
laendereinwohner = pd.read_csv('population-figures-by-country-csv_csv.csv')
laendereinwohner = laendereinwohner[['Country','Year_2016']]
laendereinwohner = laendereinwohner.fillna(0)
laendereinwohner['Year_2016'] = laendereinwohner['Year_2016'].astype(int)

In [8]:
bundeslandeinwohner = pd.read_csv('bundeslandeinw.csv',skiprows= range(1,2), nrows = 16, thousands = '.')

In [9]:
kreise_zuwachs = kreise_zuwachs.sort_values(['location_label','publication_datetime'])
kreise_zuwachs['täglicher Wachstum'] = kreise_zuwachs['cases'].pct_change()

'AL'

In [207]:
counties = pd.read_csv('covid_confirmed_usafacts.csv')
areacounty = pd.read_excel('LND01.xls')
pop = pd.read_csv('covid_county_population_usafacts.csv')

counties = counties.drop_duplicates(subset =["County Name"])
counties = counties.drop(['stateFIPS','countyFIPS'], axis=1)
arraycounties = [counties['County Name'].tolist()] + [counties['State'].tolist()]
#arraycounties = list(dict.fromkeys(arraycounties))  #Entfernt alle Dubletten
#arraycounties = sorted(arraycounties)
dflist = counties['County Name'].tolist()
dflist = list(dict.fromkeys(dflist))
dic = {}

for i in dflist:
    dic[i] = pd.DataFrame()

k = 0
avgwachstum = []

for name, df in dic.items():
        dic[name] = counties.loc[(counties['County Name'] == arraycounties[0][k]) & (counties['State'] == arraycounties[1][k])]
        dic[name] = dic[name].reset_index()
        dic[name] = dic[name].T
        dic[name] = dic[name].rename(columns={0:'cases'})
        dic[name]['County Name'] = arraycounties[0][k]
        dic[name] = dic[name].iloc[3:]
        dic[name] = dic[name].reset_index()
        dic[name] = dic[name].rename(columns={'index': 'date'})
        dic[name]['date'] = pd.to_datetime(dic[name]['date'], format='%m/%d/%y')
        dic[name] = dic[name].loc[dic[name]['date']< '2020-07-01']
        dic[name]['cases'] =  dic[name]['cases'].astype(int) 
        dic[name]['dailycases'] = dic[name]['cases'].diff()
        dic[name]['dailycases'][dic[name]['dailycases'] < 0] = 0
        dic[name]['Rate'] = dic[name]['cases'].pct_change()
        dic[name]['Rate'] = dic[name]['Rate']*100
        dic[name]['Rate'] = dic[name]['Rate'].replace([np.inf, -np.inf], 0)
        dic[name] = dic[name].fillna(0)
        avgwachstum.append(dic[name]['Rate'].sum()/len(dic[name].index))
        k = k + 1

In [236]:
areacounty = pd.read_excel('LND01.xls')
areacounty = areacounty[['Areaname','LND010190D']]
areacounty['Areaname'] = areacounty['Areaname'].str[:-4]
areacounty['Areaname'] = areacounty['Areaname'] + ' County'
areacounty = areacounty.drop_duplicates(subset =["Areaname"])
areacounty = areacounty.reset_index()

,index,Areaname,LND010190D
0,0,UNITED ST County,3787425.08
1,1,ALA County,52422.94
2,2,Autauga County,604.49
3,3,Baldwin County,2027.08
4,4,Barbour County,904.59
...,...,...,...
1882,3192,Sublette County,4935.97
1883,3193,Sweetwater County,10491.73
1884,3195,Uinta County,2087.66
1885,3196,Washakie County,2242.85


In [253]:
t = {'County':dflist,'AGR':avgwachstum}
DFCounty = pd.DataFrame(t)
DFCounty['Area'] = 0.0

for i in range(len(areacounty)):
    for j in range(len(dflist)):
        if dflist[j] == areacounty['Areaname'][i]:
            DFCounty['Area'][j] = areacounty['LND010190D'][i]

DFCounty['Area'] = DFCounty['Area'] * 2.59  #Von Meilen auf Kilometer
DFCounty['Population'] = 0

for i in range(len(pop)):
    for j in range(len(dflist)):
        if dflist[j] == pop['County Name'][i]:
            DFCounty['Population'][j] = pop['population'][i]
            
DFCounty['Density'] = DFCounty['Population'] / DFCounty['Area'] 
DFCounty=DFCounty.replace([np.inf, -np.inf], np.nan)
DFCounty = DFCounty.fillna(0)
DFCounty = DFCounty[DFCounty['Density'] != 0]
DFCounty = DFCounty[DFCounty['AGR'] != 0]

In [12]:
# Erstellung von 10 DFs 5 kleine Landkreise und 5 große Landkreise

arraykreise = kreise_zuwachs['location_label'].tolist()
arraykreise = list(dict.fromkeys(arraykreise))  #Entfernt alle Dubletten
#arraykreise = ['Mettmann','Main-Taunus-Kreis','Offenbach','Esslingen','Recklinghausen','Altmarkkreis Salzwedel','Prignitz','Ostprignitz-Ruppin','Uckermark','Stendal']
arraykreise = sorted(arraykreise)
dflist = arraykreise
d = {} #dict damit können alle erstelleten DataFrames aufgerufen werden

for j in dflist:
   d[j] = pd.DataFrame()
i = 0

durchschnittlicheswachstum = []

for name, df in d.items():
        d[name] = kreise_zuwachs[kreise_zuwachs['location_label'] == arraykreise[i]]
        d[name] = d[name].drop(columns=['population','bundesland_name'])
        d[name]['dailycases'] = d[name]['cases'].diff()
        d[name]['publication_datetime'] = pd.to_datetime(d[name]['publication_datetime'])
        d[name] = d[name].sort_values(['publication_datetime'])
        d[name]['dailycases'][d[name]['dailycases'] < 0] = 0
        d[name]['Rate'] = d[name]['cases'].pct_change()
        d[name]['Rate'] = d[name]['Rate']*100
        d[name]['Rate'] = d[name]['Rate'].replace([np.inf, -np.inf], 0)
        d[name] = d[name].fillna(0)
        
        #10 DFs sind vorhanden nun muss durchschnittliche Wachstumsrate berechnet werden
        durchschnittlicheswachstum.append(d[name]['Rate'].sum()/len(d[name].index))
        i = i + 1

Hier wird mithilfe eines Dictionary für jeden Landkreis ein DF erstellt und dann eine Liste in der alle Durchschnittlichen Wachstumsraten aller Landkreise drin stehen.

Ich gehe davon aus. Dass wenn ein

In [13]:
d['Hochsauerlandkreis'].loc[6941,'dailycases'] = d['Hochsauerlandkreis'].loc[6941, 'cases']
d['Hochsauerlandkreis']['kumcases'] = d['Hochsauerlandkreis']['dailycases'].cumsum()

d['Hochsauerlandkreis']['dailydeaths'] = d['Hochsauerlandkreis']['deaths'].diff()
d['Hochsauerlandkreis']['dailydeaths'][d['Hochsauerlandkreis']['dailydeaths'] < 0] = 0
d['Hochsauerlandkreis'].loc[6941,'dailydeaths'] = d['Hochsauerlandkreis'].loc[6941, 'deaths']
d['Hochsauerlandkreis']['kumdeaths'] = d['Hochsauerlandkreis']['dailydeaths'].cumsum()


In [14]:
d['Essen'].loc[6936,'dailycases'] = d['Essen'].loc[6936, 'cases']
d['Essen']['kumcases'] = d['Essen']['dailycases'].cumsum()

d['Essen']['dailydeaths'] = d['Essen']['deaths'].diff()
d['Essen']['dailydeaths'][d['Essen']['dailydeaths'] < 0] = 0
d['Essen'].loc[6936,'dailydeaths'] = d['Essen'].loc[6936, 'deaths']
d['Essen']['kumdeaths'] = d['Essen']['dailydeaths'].cumsum()


In [15]:
d['Hochsauerlandkreis']['Wochentag'] = d['Hochsauerlandkreis']['publication_datetime'].dt.day_name()
dic['Albany County']['Weekday'] = dic['Albany County']['date'].dt.day_name()
d['Essen']['Wochentag'] = d['Essen']['publication_datetime'].dt.day_name()

In [16]:
import locale
locale.setlocale(locale.LC_ALL, '')

'German_Germany.1252'

In [17]:
#Noch in bearbeitung
#bund_zuwachsraw = pd.read_csv('COVID-19 Cases Germany_1591375436907.csv')
#bund_zuwachsraw.drop(['deaths_per_100k','deaths','relative_case_changes','fb_id','cases_per_population','relative_death_changes','bundesland_ags','kreis_ags','kreis_nuts','kreis_name','deaths_per_population','fb_datetime'], axis=1, inplace=True)
#bund_zuwachsraw = bund_zuwachsraw.fillna(0)
#bund_zuwachsraw['publication_datetime'] = bund_zuwachsraw['publication_datetime'].str[:11]
#bund_zuwachsraw = bund_zuwachsraw.sort_values(['publication_datetime'])

In [18]:
#Noch in bearbeitung
#bundzuwachsraw = pd.read_csv('RKI_COVID19.csv')
#listSH = ["Schleswig-Holstein"]
#SH = bundzuwachsraw.loc[bundzuwachsraw['Bundesland'].isin(listSH)]
#SH['Meldedatum'] = pd.to_datetime(SH['Meldedatum'])

In [19]:
#Noch in bearbeitung
#import datetime

#date = datetime.datetime(2020, 6, 30)
#SHsum = 0

#for i in SH.index:
 #   if SH['Meldedatum'][i] == date:
 #       SHsum = SHsum + SH['AnzahlFall'][i]
 #   #date = date - timedelta(days=1)
        
#print(date)

In [20]:
from string import ascii_lowercase

bundzuwachsraw = pd.read_csv('table-1.csv')
bundzuwachsraw.set_index('Datum')
bundzuwachsraw = bundzuwachsraw.drop(['KW','Differenz zum Vortag','Bem.','Datum'],1) 
bundzuwachsraw = bundzuwachsraw.iloc[:-1]
bundzuwachsraw = bundzuwachsraw.fillna(0)
bundzuwachsraw = bundzuwachsraw.replace('—',0)

for column in bundzuwachsraw:
    bundzuwachsraw[column] = bundzuwachsraw[column].astype(str)
    bundzuwachsraw[column] = bundzuwachsraw[column].str.replace('.','')
    for c in ascii_lowercase:
        bundzuwachsraw[column] = bundzuwachsraw[column].str.replace(c,'')
    bundzuwachsraw[column] = bundzuwachsraw[column].str.replace(')','')
    bundzuwachsraw[column] = bundzuwachsraw[column].str.replace('(','')
    bundzuwachsraw[column] = bundzuwachsraw[column].astype(int)    

In [21]:
bundzuwachs = bundzuwachsraw.diff()
bundzuwachs.iloc[0] = bundzuwachsraw.iloc[0]
bundzuwachsraw['Date'] = pd.date_range(start='24/2/2020', periods=len(bundzuwachsraw), freq='D')        
bundzuwachs['Date'] = pd.date_range(start='24/2/2020', periods=len(bundzuwachsraw), freq='D')
bundzuwachsraw = bundzuwachsraw.set_index('Date')
bundzuwachs = bundzuwachs.set_index('Date')

In [22]:
bundchange = bundzuwachsraw.pct_change()
bundchange = bundchange*100
bundchange = bundchange.fillna(0)
bundchange = bundchange.replace([np.inf, -np.inf], 0) # Was sollte am besten hier hin für die Wachstumsrate?
# tägliche prozentuale Änderung der Coronazahlen

In [23]:
bundzuwachsraw_1 = bundzuwachsraw.iloc[:128] # Februar Tage + März + April + Mai + Juni
bundzuwachsraw_2 = bundzuwachsraw.iloc[128:]
bundzuwachs_1 = bundzuwachs.iloc[:128]
bundzuwachs_2 = bundzuwachs.iloc[128:]
bundchange_1 = bundchange.iloc[:128]
bundchange_2 = bundchange.iloc[128:]

In [24]:
durchschnittlichwachstum = bundchange_1.sum()/len(bundchange_1.index)
durchschnittlichwachstum = bundchange_1.sum()/len(bundchange_1.index)
durchwachstumfnl = pd.DataFrame({'Bundesland':durchschnittlichwachstum.index,'AGR':durchschnittlichwachstum.values})
#fnl = Final  AGR = average growth rate
liste = ['Baden-Württemberg','Bayern','Berlin','Brandenburg','Bremen','Hamburg','Hessen','Mecklenburg-Vorpommern','Niedersachsen','Nordrhein-Westfalen','Rheinland-Pfalz','Saarland','Sachsen','Sachsen-Anhalt','Schleswig-Holstein','Thüringen','Gesamt']

for i, row in durchwachstumfnl.iterrows():
        durchwachstumfnl.at[i,'Bundesland'] = liste[i]

In [25]:
dichte = pd.read_excel('A_BevDichte_Kreise.xlsx')
dichte = dichte.drop(columns = ['Kennziffer','Thünen-Typ'])
dichte = dichte.sort_values(by=['Name'])
dichte = dichte.reset_index()
dichte = dichte.drop(columns = 'index')

#Wird genutzt um die Stadt von dem Landkreis zu trennen
name = ' Stadt'

for j in range(len(dichte)):
    if j == 401:
        break
    else:
        if dichte['Name'][j] == dichte['Name'][j+1]:
            if dichte['Bevölkerungsdichte 2014'][j] > dichte['Bevölkerungsdichte 2014'][j+1]:
                dichte['Name'][j+1] = dichte['Name'][j+1] + name
            else:
                dichte['Name'][j] = dichte['Name'][j] + name

In [26]:
dichteU = pd.DataFrame(columns= ['Landkreis/Kreis','Bev.D.Ew./km²'])
dichteU['Landkreis/Kreis'] = arraykreise
dichteU = dichteU.fillna(0)

for i in range(len(dichte)):
    for j in range(len(arraykreise)):
        if arraykreise[j] == dichte['Name'][i]:
            dichteU['Bev.D.Ew./km²'][j] = dichte['Bevölkerungsdichte 2014'][i]

In [27]:
t = {'Landkreise/Kreis':arraykreise,'AGR':durchschnittlicheswachstum}
compLK = pd.DataFrame(t) #compLK = Vergleich der Landkreise von Wachstum und Bevölkerungsdichte
compLK['Bev.D.Ew./km²'] = dichteU['Bev.D.Ew./km²'].astype(int)
compLK = compLK[compLK['Bev.D.Ew./km²'] != 0]
#Sonst geht Clustering nicht
compLKfnl = compLK.drop(columns =['Landkreise/Kreis'])
compLKfnl = compLKfnl.rename(columns={'Bev.D.Ew./km²':'Dichte'})

,AGR,Dichte
0,2.530475,161
1,4.399501,166
2,7.676924,139
3,6.611777,163
4,6.688335,200
...,...,...
408,7.372777,121
410,4.011612,1418
411,5.618100,165
412,5.484557,203
